# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 50

### Elementos do Grupo

Número: 53841   Nome: Tomás Carrilho     
Número: 54838   Nome: Matilde Carvalho    
Número: 54859   Nome: Rita Rodrigues     

## Representação de variáveis, domínios, vizinhos e restrições

Descreva aqui, textualmente, como decidiu representar as variáveis, domínios, vizinhos e restrições em Python;

Daqui em diante consideramos que o quadrado é representado numa matriz *n x n*.

### Quadrado latino

Vamos representar as entradas que estão preenchidas inicialmente por uma lista de strings.
Consideremos, por exemplo, a seguinte matriz. \
$\begin{bmatrix}
1 &   & 3 \\
  & 3 &   \\
  &   & 2
\end{bmatrix}$ \
A lista correspondente seria:
```python
['x11 = 1', 'x13 = 3', 'x22 = 3', 'x33 = 2']
```

#### Definição das variáveis
$X_{ij}$ correspondente à célula na linha $i$ e na coluna $j$; $i,j \in \{1, ..., n\}$ \
Em python, vamos ter a lista
``` python
    variaveis = ['x11', ..., 'xnn']
```
Que pode ser gerada da seguinte forma
``` python
    variaveis = [('x' + str(i) + str(j)) for i in range(1,n+1) for j in range(1,n+1)]
```

#### Definição dos domínios
O domínio é um dicionário em que as chaves são as variáveis e os valores são o domínio de cada variável do problema. Estes domínios deveriam ser listas de 1 a *n*, mas dado que a implementação em CSP assume que as restrições são binárias, as restrições unárias foram eliminadas.
``` python
    dominios = {}
    for v in variaveis :
        dominios[v] = [i for i in range(1, n+1)]
        
    # No próximo ciclo for vamos eliminar as restrições unárias
    
    for restricao in lista_entradas:
        dominios[restricao.split()[0]] = [int(restricao.split()[2])]
```

#### Definição dos vizinhos
Vamos criar o grafo de restrições com os arcos seguintes:

    x11 : x12 ... x1n x21 ... xn1
    x12 : x13 ... x1n x22 ... xn2
    ...
    x21 : x22 ... x2n x31 ... xn1
    ...

``` python
    viz = {}
    for v in variaveis:
        viz[v] = []
        for k in range(1, n+1):
            if k > int(v[1]):
                viz[v].append('x' + str(k) + v[2])
            if k > int(v[2]):
                viz[v].append('x' + v[1] + str(k))

    vi = ''
    for i in viz:
        vi += str(i)+ ' : '
        for j in viz[i]:
            vi += j + ' '
        vi += "; "

    vizinhos = parse_neighbors(vi[:-2])
```

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Vamos definir uma função que verifica se duas variáveis X, e Y satisfazem todas as restrições binárias em que estão envolvidas, quando afetadas respetivamente aos valores a e b. \
Temos as seguintes restrições não binárias (globais):
- R_G1) n allDiff para cada linha
- R_G2) n allDiff para cada coluna 

Que terão de ser transformadas em binárias. 
``` python
    def diferentes(x,y):
        return x != y
```
Vamos guardar as restrições binárias num dicionário em que cada chave é um par de variáveis e o valor é uma restrição (função).
``` python
    restricoes = {}

    for v in variaveis:
        for k in range(1, n+1):
            if str(k) == v[1]:
                for j in range(1, n+1):
                    if str(j) != v[2]:
                        restricoes[(v, 'x' + v[1] + str(j))] = diferentes
            if str(k) == v[2]:
                for j in range(1, n+1):
                    if str(j) != v[1]:
                        restricoes[(v, 'x' + str(j) + v[2])] = diferentes

```
No final da geração das restrições, é necessário verificar se X=a e Y=b satisfazem a restrição, que depende da ordem do par. Se não houver uma restrição entre essas duas variáveis então o resultado é True. \
Assim, a função `restricoes(X, a, Y, b)`, depois de se terem definido as condições e o dicionário com as restrições, irá devolver:
```python
    if (X,Y) in restricoes :
        return restricoes[(X,Y)](a,b)
    else: # Se não há restrição
        True
```

### Futoshiki

As variáveis, os domínios e as restrições são os mesmos.
Surge ainda outra restrição, caso duas células adjacentes contenham uma desigualdade: nesse caso tem de se verificar a desigualdade.
As desigualdades são representadas por uma lista de desigualdades, por exemplo: 
``` python
['x13 > x14', 'x32 < x33', 'x32 > x42']
```

#### Definir a função  que verifica a satisfação das restrições entre duas variáveis afectadas
Temos de adicionar o seguinte:
``` python
    def menor(x,y):
        return x < y
    def maior(x,y):
        return x > y
    
    restricoes = {}
    for restricao in lista_desigualdades:
        if restricao.split()[1] == '<':
            restricoes[(restricao.split()[0],restricao.split()[2])] = menor
            restricoes[(restricao.split()[2],restricao.split()[0])] = maior
        if restricao.split()[1] == '>':
            restricoes[(restricao.split()[0],restricao.split()[2])] = maior
            restricoes[(restricao.split()[2],restricao.split()[0])] = menor
```

## Formulação do problema

### Quadrado latino

In [1]:
from csp import *

def diferentes(x,y):
    """x diferente de y"""
    return x != y

def quadrado_latino(n, lista_entradas):
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    
    # Definir Variáveis
    variaveis = [('x' + str(i) + str(j)) for i in range(1,n+1) for j in range(1,n+1)]    
    
    # Definir Domínios
    dominios = {}
    for v in variaveis :
        dominios[v] = [i for i in range(1, n+1)]
        
    # No próximo ciclo for vamos eliminar as restrições unárias
    
    for restricao in lista_entradas:
        dominios[restricao.split()[0]] = [int(restricao.split()[2])]
        
    # Definir Vizinhos
    viz = {}
    for v in variaveis:
        viz[v] = []
        for k in range(1, n+1):
            if k > int(v[1]):
                viz[v].append('x' + str(k) + v[2])
            if k > int(v[2]):
                viz[v].append('x' + v[1] + str(k))

    vi = ''
    for i in viz:
        vi += str(i)+ ' : '
        for j in viz[i]:
            vi += j + ' '
        vi += "; "

    vizinhos = parse_neighbors(vi[:-2])
                
    #Definir função que verifica restrições binárias
    def restricoes_quadrado_latino(X, a, Y, b):
        restricoes = {}
        
        for v in variaveis:
            for k in range(1, n+1):
                if str(k) == v[1]:
                    for j in range(1, n+1):
                        if str(j) != v[2]:
                            restricoes[(v, 'x' + v[1] + str(j))] = diferentes
                if str(k) == v[2]:
                    for j in range(1, n+1):
                        if str(j) != v[1]:
                            restricoes[(v, 'x' + str(j) + v[2])] = diferentes

        if (X,Y) in restricoes:
            return restricoes[(X,Y)](a,b)
        else:
            True

    return CSP(variaveis, dominios, vizinhos, restricoes_quadrado_latino)

### Futoshiki

In [2]:
# Funções binárias usadas nas restrições
def diferentes(x,y):
    return x != y

def menor(x,y):
    return x < y

def maior(x,y):
    return x > y


def futoshiki(n, lista_entradas, lista_desigualdades):
    
    # Definir Variáveis
    variaveis = [('x' + str(i) + str(j)) for i in range(1,n+1) for j in range(1,n+1)]    
    
    # Definir Domínios
    dominios = {}
    for v in variaveis :
        dominios[v] = [i for i in range(1, n+1)]
        
    # No próximo ciclo for vamos eliminar as restrições unárias
    
    for restricao in lista_entradas:
        dominios[restricao.split()[0]] = [int(restricao.split()[2])]
        
    # Definir Vizinhos
    viz = {}
    for v in variaveis:
        viz[v] = []
        for k in range(1, n+1):
            if k > int(v[1]):
                viz[v].append('x' + str(k) + v[2])
            if k > int(v[2]):
                viz[v].append('x' + v[1] + str(k))

    vi = ''
    for i in viz:
        vi += str(i)+ ' : '
        for j in viz[i]:
            vi += j + ' '
        vi += "; "

    vizinhos = parse_neighbors(vi[:-2])
    
    #Definir função que verifica restrições binárias
    def restricoes_futoshiki(X, a, Y, b):        
        
        restricoes = {}
        
        for v in variaveis:
            for k in range(1, n+1):
                if str(k) == v[1]:
                    for j in range(1, n+1):
                        if str(j) != v[2]:
                            restricoes[(v, 'x' + v[1] + str(j))] = diferentes
                if str(k) == v[2]:
                    for j in range(1, n+1):
                        if str(j) != v[1]:
                            restricoes[(v, 'x' + str(j) + v[2])] = diferentes
            
        
        
        # restrições binárias
        for restricao in lista_desigualdades:
            if restricao.split()[1] == '<':
                restricoes[(restricao.split()[0],restricao.split()[2])] = menor
                restricoes[(restricao.split()[2],restricao.split()[0])] = maior
            if restricao.split()[1] == '>':
                restricoes[(restricao.split()[0],restricao.split()[2])] = maior
                restricoes[(restricao.split()[2],restricao.split()[0])] = menor

        if (X,Y) in restricoes:
            return restricoes[(X,Y)](a,b)
        else:
            True
            
    return CSP(variaveis, dominios, vizinhos, restricoes_futoshiki)

Para criar um quadrado latino basta `quadrado_latino(número da dimensão, [restrições separadas por virgulas])`

Para criar um Futoshiki basta `futoshiki(número da dimensão, [restrições de igualdade separadas por virgulas], [restrições de maior e menor separadas por virgulas])`

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

#### Problema de quadrado latino *4x4* sem quadrados preenchidos inicialmente

In [3]:
p1 = quadrado_latino(4, [])

In [4]:
print("Variáveis = ", p1.variables)
print("Domínios = ", p1.domains)
print("Vizinhos = ", p1.neighbors)

Variáveis =  ['x11', 'x12', 'x13', 'x14', 'x21', 'x22', 'x23', 'x24', 'x31', 'x32', 'x33', 'x34', 'x41', 'x42', 'x43', 'x44']
Domínios =  {'x11': [1, 2, 3, 4], 'x12': [1, 2, 3, 4], 'x13': [1, 2, 3, 4], 'x14': [1, 2, 3, 4], 'x21': [1, 2, 3, 4], 'x22': [1, 2, 3, 4], 'x23': [1, 2, 3, 4], 'x24': [1, 2, 3, 4], 'x31': [1, 2, 3, 4], 'x32': [1, 2, 3, 4], 'x33': [1, 2, 3, 4], 'x34': [1, 2, 3, 4], 'x41': [1, 2, 3, 4], 'x42': [1, 2, 3, 4], 'x43': [1, 2, 3, 4], 'x44': [1, 2, 3, 4]}
Vizinhos =  defaultdict(<class 'list'>, {'x11': ['x21', 'x12', 'x31', 'x13', 'x41', 'x14'], 'x21': ['x11', 'x22', 'x31', 'x23', 'x41', 'x24'], 'x12': ['x11', 'x22', 'x32', 'x13', 'x42', 'x14'], 'x31': ['x11', 'x21', 'x32', 'x33', 'x41', 'x34'], 'x13': ['x11', 'x12', 'x23', 'x33', 'x43', 'x14'], 'x41': ['x11', 'x21', 'x31', 'x42', 'x43', 'x44'], 'x14': ['x11', 'x12', 'x13', 'x24', 'x34', 'x44'], 'x22': ['x12', 'x21', 'x32', 'x23', 'x42', 'x24'], 'x32': ['x12', 'x22', 'x31', 'x33', 'x42', 'x34'], 'x42': ['x12', 'x22', 'x3

#### Problema de quadrado latino *4x4* com quadrados preenchidos inicialmente

In [5]:
p2 = quadrado_latino(4, ['x11 = 2', 'x31 = 3', 'x32 = 1', 'x41 = 1', 'x42 = 2', 'x43 = 3'])

In [6]:
print("Variáveis = ", p2.variables)
print("Domínios = ", p2.domains)
print("Vizinhos = ", p2.neighbors)

Variáveis =  ['x11', 'x12', 'x13', 'x14', 'x21', 'x22', 'x23', 'x24', 'x31', 'x32', 'x33', 'x34', 'x41', 'x42', 'x43', 'x44']
Domínios =  {'x11': [2], 'x12': [1, 2, 3, 4], 'x13': [1, 2, 3, 4], 'x14': [1, 2, 3, 4], 'x21': [1, 2, 3, 4], 'x22': [1, 2, 3, 4], 'x23': [1, 2, 3, 4], 'x24': [1, 2, 3, 4], 'x31': [3], 'x32': [1], 'x33': [1, 2, 3, 4], 'x34': [1, 2, 3, 4], 'x41': [1], 'x42': [2], 'x43': [3], 'x44': [1, 2, 3, 4]}
Vizinhos =  defaultdict(<class 'list'>, {'x11': ['x21', 'x12', 'x31', 'x13', 'x41', 'x14'], 'x21': ['x11', 'x22', 'x31', 'x23', 'x41', 'x24'], 'x12': ['x11', 'x22', 'x32', 'x13', 'x42', 'x14'], 'x31': ['x11', 'x21', 'x32', 'x33', 'x41', 'x34'], 'x13': ['x11', 'x12', 'x23', 'x33', 'x43', 'x14'], 'x41': ['x11', 'x21', 'x31', 'x42', 'x43', 'x44'], 'x14': ['x11', 'x12', 'x13', 'x24', 'x34', 'x44'], 'x22': ['x12', 'x21', 'x32', 'x23', 'x42', 'x24'], 'x32': ['x12', 'x22', 'x31', 'x33', 'x42', 'x34'], 'x42': ['x12', 'x22', 'x32', 'x41', 'x43', 'x44'], 'x23': ['x13', 'x21', 'x22',

Quer no quadrado latino simples `p1` quer o quadrado latino com restrições `p2` no número de variáveis não se altera. Relativamente aos domínios no quadrado latino simples, como não tem restrições todas as variáveis têm o mesmo domínio com a dimensão do quadrado, neste caso 4, ou seja, as variáveis podem tomar os valores pertencentes ao domínio [1,2,3,4].  Por outro lado, como o quadrado latino `p2` tem restrições, como por exemplo `x11=2`, assim a variável `x11` só pode tomar o valor 2, o que resulta numa restrição ao domínio de `x11`. Relativamente aos vizinhos de cada quadrado podemos observar que são iguais. Neste caso em concreto as restrições apenas traduzem uma mudança nos domínios das variáveis. 

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

###  Procura com retrocesso, simples, sem inferência nem heurísticas

In [7]:
import timeit

In [8]:
dt={}

In [9]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r) 
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso, simples, sem inferência nem heurísticas"]=time

Solução para quadrado latino 4x4 =  {'x11': 1, 'x12': 2, 'x13': 3, 'x14': 4, 'x21': 2, 'x22': 1, 'x23': 4, 'x24': 3, 'x31': 3, 'x32': 4, 'x33': 1, 'x34': 2, 'x41': 4, 'x42': 3, 'x43': 2, 'x44': 1}
Time:  0.02444040000000003


Para percebermos detalhadamente todo o processo de resolução vamos colocar `verbose=True`.

In [10]:
p = quadrado_latino(4, [])
r = backtracking_search(p,verbose = True)
print('Solução para quadrado latino 4x4 = ',r)

Curr_domains: None
Current assignment: {}

Next selected Var = x11
Sorted domain left [1, 2, 3, 4]
--Test x11 1
----Assigned!
Curr_domains: {'x11': [1], 'x12': [1, 2, 3, 4], 'x13': [1, 2, 3, 4], 'x14': [1, 2, 3, 4], 'x21': [1, 2, 3, 4], 'x22': [1, 2, 3, 4], 'x23': [1, 2, 3, 4], 'x24': [1, 2, 3, 4], 'x31': [1, 2, 3, 4], 'x32': [1, 2, 3, 4], 'x33': [1, 2, 3, 4], 'x34': [1, 2, 3, 4], 'x41': [1, 2, 3, 4], 'x42': [1, 2, 3, 4], 'x43': [1, 2, 3, 4], 'x44': [1, 2, 3, 4]}
Current assignment: {'x11': 1}

Next selected Var = x12
Sorted domain left [1, 2, 3, 4]
--Test x12 1
----Conflict!!
--Test x12 2
----Assigned!
Curr_domains: {'x11': [1], 'x12': [2], 'x13': [1, 2, 3, 4], 'x14': [1, 2, 3, 4], 'x21': [1, 2, 3, 4], 'x22': [1, 2, 3, 4], 'x23': [1, 2, 3, 4], 'x24': [1, 2, 3, 4], 'x31': [1, 2, 3, 4], 'x32': [1, 2, 3, 4], 'x33': [1, 2, 3, 4], 'x34': [1, 2, 3, 4], 'x41': [1, 2, 3, 4], 'x42': [1, 2, 3, 4], 'x43': [1, 2, 3, 4], 'x44': [1, 2, 3, 4]}
Current assignment: {'x11': 1, 'x12': 2}

Next selected 

### Procura com retrocesso com forward checking com pré-filtragem por AC3

In [11]:
p = quadrado_latino(4, [])
print("\nRetrocesso com forward checking com pré-filtragem por AC3")
p_ac3 = AC3(p)
start = timeit.default_timer()
r = backtracking_search(p_ac3,inference = forward_checking)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com forward checking com pré-filtragem por AC3"]=time


Retrocesso com forward checking com pré-filtragem por AC3
Solução para quadrado latino 4x4 =  {'x11': 1, 'x12': 2, 'x13': 3, 'x14': 4, 'x21': 2, 'x22': 1, 'x23': 4, 'x24': 3, 'x31': 3, 'x32': 4, 'x33': 1, 'x34': 2, 'x41': 4, 'x42': 3, 'x43': 2, 'x44': 1}
Time:  0.030144599999999855


In [12]:
# Vejemos os domínios após o AC3
print('\nDomínio após AC3:',p_ac3.domains)


Domínio após AC3: {'x11': [1, 2, 3, 4], 'x12': [1, 2, 3, 4], 'x13': [1, 2, 3, 4], 'x14': [1, 2, 3, 4], 'x21': [1, 2, 3, 4], 'x22': [1, 2, 3, 4], 'x23': [1, 2, 3, 4], 'x24': [1, 2, 3, 4], 'x31': [1, 2, 3, 4], 'x32': [1, 2, 3, 4], 'x33': [1, 2, 3, 4], 'x34': [1, 2, 3, 4], 'x41': [1, 2, 3, 4], 'x42': [1, 2, 3, 4], 'x43': [1, 2, 3, 4], 'x44': [1, 2, 3, 4]}


Neste problema o AC3 não reduz os domínios das variáveis.

### Procura com retrocesso com a heurística da variável com menor domínio (mrv)

In [13]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p,select_unassigned_variable = mrv)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com a heurística da variável com menor domínio (mrv)"]=time

Solução para quadrado latino 4x4 =  {'x24': 1, 'x42': 1, 'x14': 2, 'x13': 1, 'x32': 2, 'x21': 2, 'x43': 2, 'x22': 3, 'x31': 1, 'x34': 4, 'x23': 4, 'x33': 3, 'x11': 3, 'x41': 4, 'x44': 3, 'x12': 4}
Time:  0.03930509999999998


### Procura com retrocesso com forward checking e sem heurísticas

In [14]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p,inference=forward_checking)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com forward checking e sem heurísticas"]=time

Solução para quadrado latino 4x4 =  {'x11': 1, 'x12': 2, 'x13': 3, 'x14': 4, 'x21': 2, 'x22': 1, 'x23': 4, 'x24': 3, 'x31': 3, 'x32': 4, 'x33': 1, 'x34': 2, 'x41': 4, 'x42': 3, 'x43': 2, 'x44': 1}
Time:  0.03418510000000019


### Procura com retrocesso com forward checking e com mrv

In [15]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p,select_unassigned_variable = mrv,inference=forward_checking)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com forward checking e com mrv"]=time

Solução para quadrado latino 4x4 =  {'x41': 1, 'x21': 2, 'x31': 3, 'x11': 4, 'x22': 1, 'x23': 3, 'x24': 4, 'x32': 2, 'x34': 1, 'x12': 3, 'x42': 4, 'x43': 2, 'x33': 4, 'x14': 2, 'x44': 3, 'x13': 1}
Time:  0.03289680000000006


### Procura com retrocesso com mac

In [16]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p,inference=mac)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com mac"]=time

Solução para quadrado latino 4x4 =  {'x11': 1, 'x12': 2, 'x13': 3, 'x14': 4, 'x21': 2, 'x22': 1, 'x23': 4, 'x24': 3, 'x31': 3, 'x32': 4, 'x33': 1, 'x34': 2, 'x41': 4, 'x42': 3, 'x43': 2, 'x44': 1}
Time:  0.0962632000000001


###  Procura com retrocesso com mac e com mrv

In [17]:
p = quadrado_latino(4, [])
start = timeit.default_timer()
r = backtracking_search(p,select_unassigned_variable = mrv,inference=mac)
stop = timeit.default_timer()
print('Solução para quadrado latino 4x4 = ',r)
time = stop-start
print('Time: ', time)

dt["Procura com retrocesso com mac e com mrv"]=time

Solução para quadrado latino 4x4 =  {'x42': 1, 'x22': 2, 'x12': 3, 'x32': 4, 'x44': 2, 'x14': 1, 'x34': 3, 'x24': 4, 'x41': 3, 'x13': 2, 'x23': 3, 'x21': 1, 'x33': 1, 'x31': 2, 'x11': 4, 'x43': 4}
Time:  0.09478200000000037


Assim sendo ordenado de forma crescente pelo tempo gasto a resolver o problema:

In [18]:
for item in sorted(dt, key = dt.get):
    print (item, dt[item])

Procura com retrocesso, simples, sem inferência nem heurísticas 0.02444040000000003
Procura com retrocesso com forward checking com pré-filtragem por AC3 0.030144599999999855
Procura com retrocesso com forward checking e com mrv 0.03289680000000006
Procura com retrocesso com forward checking e sem heurísticas 0.03418510000000019
Procura com retrocesso com a heurística da variável com menor domínio (mrv) 0.03930509999999998
Procura com retrocesso com mac e com mrv 0.09478200000000037
Procura com retrocesso com mac 0.0962632000000001


#### Quadrado latino *5x5*

In [19]:
dt3={}
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
print('Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas')
start = timeit.default_timer()
r0 = backtracking_search(p3)
stop = timeit.default_timer()
time0 = stop-start
print('Solução:',r0)
print('Time: ', time0)
dt3["Procura com retrocesso, simples, sem inferência nem heurísticas"]=time0

print("\nRetrocesso com forward checking com pré-filtragem por AC3")
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
p_ac3 = AC3(p3)
r1 = backtracking_search(p_ac3,inference = forward_checking)
stop = timeit.default_timer()
time1 = stop-start
print('Solução:',r1)
print('Time: ', time1)
dt3["Procura com retrocesso com forward checking com pré-filtragem por AC3"]=time1

print('\nApliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):')
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
r2 = backtracking_search(p3,select_unassigned_variable = mrv)
stop = timeit.default_timer()
time2 = stop-start
print('Solução:',r2)
print('Time: ', time2)
dt3["Procura com retrocesso com a heurística da variável com menor domínio (mrv)"]=time2

print('\nApliquemos a procura com retrocesso com forward checking e sem heurísticas')
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
r3 = backtracking_search(p3,inference=forward_checking)
stop = timeit.default_timer()
time3 = stop-start
print('Solução:',r3)
print('Time: ', time3)
dt3["Procura com retrocesso com forward checking e sem heurísticas"]=time3

print('\nApliquemos a procura com retrocesso com forward checking e com mrv')
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
r4 = backtracking_search(p3,select_unassigned_variable = mrv,inference=forward_checking)
stop = timeit.default_timer()
time4 = stop-start
print('Solução:',r4)
print('Time: ', time4)
dt3["Procura com retrocesso com forward checking e com mrv"]=time4

print('\nApliquemos a procura com retrocesso com mac')
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
r5 = backtracking_search(p3,inference=mac)
stop = timeit.default_timer()
time5 = stop-start
print('Solução:',r5)
print('Time: ', time5)
dt3["Procura com retrocesso com mac"]=time5

print('\nApliquemos a procura com retrocesso com mac e com mrv')
p3 = quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])
start = timeit.default_timer()
r6 = backtracking_search(p3,select_unassigned_variable = mrv,inference=mac)
stop = timeit.default_timer()
time6 = stop-start
print('Solução:',r6)
print('Time: ', time6)
dt3["Procura com retrocesso com mac e com mrv"]=time6

Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x15': 5, 'x21': 2, 'x22': 3, 'x23': 1, 'x24': 5, 'x25': 4, 'x31': 1, 'x32': 2, 'x33': 5, 'x34': 4, 'x35': 3, 'x41': 5, 'x42': 4, 'x43': 3, 'x44': 2, 'x45': 1, 'x51': 3, 'x52': 5, 'x53': 4, 'x54': 1, 'x55': 2}
Time:  0.4194906999999999

Retrocesso com forward checking com pré-filtragem por AC3
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x15': 5, 'x21': 2, 'x22': 3, 'x23': 1, 'x24': 5, 'x25': 4, 'x31': 1, 'x32': 2, 'x33': 5, 'x34': 4, 'x35': 3, 'x41': 5, 'x42': 4, 'x43': 3, 'x44': 2, 'x45': 1, 'x51': 3, 'x52': 5, 'x53': 4, 'x54': 1, 'x55': 2}
Time:  0.3162351000000001

Apliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):
Solução: {'x11': 4, 'x35': 3, 'x23': 1, 'x42': 4, 'x55': 2, 'x52': 1, 'x53': 3, 'x54': 4, 'x25': 4, 'x44': 1, 'x41': 3, 'x13': 5, 'x32': 5, 'x12': 2, 'x24': 5, 'x14': 3, 'x45': 5, 'x31': 1, 'x43': 2, '

Assim sendo ordenado de forma crescente pelo tempo gasto a resolver o problema:

In [20]:
for item in sorted(dt3, key = dt3.get):
    print (item, dt3[item])

Procura com retrocesso com forward checking e com mrv 0.11102349999999994
Procura com retrocesso com forward checking e sem heurísticas 0.1215862999999997
Procura com retrocesso com forward checking com pré-filtragem por AC3 0.3162351000000001
Procura com retrocesso com mac 0.39965410000000023
Procura com retrocesso, simples, sem inferência nem heurísticas 0.4194906999999999
Procura com retrocesso com mac e com mrv 0.43039999999999967
Procura com retrocesso com a heurística da variável com menor domínio (mrv) 0.9115991000000001


## Visualização do problema

In [21]:
# Implemente uma função que permita visualizar o quadrado latino, antes e depois de resolvido.
def display_quadrado_latino(n, lista_entradas):
    variaveis = [('x' + str(i) + str(j)) for i in range(1,n+1) for j in range(1,n+1)]
    print('Quadrado latino antes de estar resolvido')
    barra = '----' * n
    print( ' '+barra[:-1] +' ')
    for v in variaveis:
        if variaveis.index(v)%n==0:
            print('| ',end='')
        conteudo = ' '
        separacao = ' | '
        for restricoes in lista_entradas:
            if v in restricoes.split()[0]:
                conteudo = restricoes.split()[2]
        print(conteudo, end = separacao)
        if variaveis.index(v)%n==n-1:
            linha = ''
            for c in range(1, n+1):
                adicionar_linha = '----'
                linha += adicionar_linha
            print()
            print(' ' + linha[:-1] + ' ')
    r = backtracking_search(quadrado_latino(n, lista_entradas))
    print('Quadrado latino depois de estar resolvido')
    barra = '----' * n
    print( ' '+barra[:-1] +' ')
    for v in variaveis:
        if variaveis.index(v)%n==0:
            print('| ',end='')
        conteudo = r[v]
        separacao = ' | '
        print(conteudo, end = separacao)
        if variaveis.index(v)%n==n-1:
            linha = ''
            for c in range(1, n+1):
                adicionar_linha = '----'
                linha += adicionar_linha
            print()
            print(' ' + linha[:-1] + ' ')

In [22]:
display_quadrado_latino(4, ['x11 = 2', 'x31 = 3', 'x32 = 1', 'x41 = 1', 'x42 = 2', 'x43 = 3'])

Quadrado latino antes de estar resolvido
 --------------- 
| 2 |   |   |   | 
 --------------- 
|   |   |   |   | 
 --------------- 
| 3 | 1 |   |   | 
 --------------- 
| 1 | 2 | 3 |   | 
 --------------- 
Quadrado latino depois de estar resolvido
 --------------- 
| 2 | 4 | 1 | 3 | 
 --------------- 
| 4 | 3 | 2 | 1 | 
 --------------- 
| 3 | 1 | 4 | 2 | 
 --------------- 
| 1 | 2 | 3 | 4 | 
 --------------- 


In [23]:
display_quadrado_latino(5, ['x11 = 4', 'x23 = 1', 'x35 = 3', 'x42 = 4', 'x55 = 2'])

Quadrado latino antes de estar resolvido
 ------------------- 
| 4 |   |   |   |   | 
 ------------------- 
|   |   | 1 |   |   | 
 ------------------- 
|   |   |   |   | 3 | 
 ------------------- 
|   | 4 |   |   |   | 
 ------------------- 
|   |   |   |   | 2 | 
 ------------------- 
Quadrado latino depois de estar resolvido
 ------------------- 
| 4 | 1 | 2 | 3 | 5 | 
 ------------------- 
| 2 | 3 | 1 | 5 | 4 | 
 ------------------- 
| 1 | 2 | 5 | 4 | 3 | 
 ------------------- 
| 5 | 4 | 3 | 2 | 1 | 
 ------------------- 
| 3 | 5 | 4 | 1 | 2 | 
 ------------------- 


## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

In [24]:
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])

In [25]:
print("Variáveis = ", p4.variables)
print("Domínios = ", p4.domains)
print("Vizinhos = ", p4.neighbors)

Variáveis =  ['x11', 'x12', 'x13', 'x14', 'x15', 'x21', 'x22', 'x23', 'x24', 'x25', 'x31', 'x32', 'x33', 'x34', 'x35', 'x41', 'x42', 'x43', 'x44', 'x45', 'x51', 'x52', 'x53', 'x54', 'x55']
Domínios =  {'x11': [1, 2, 3, 4, 5], 'x12': [1, 2, 3, 4, 5], 'x13': [1, 2, 3, 4, 5], 'x14': [1, 2, 3, 4, 5], 'x15': [3], 'x21': [1, 2, 3, 4, 5], 'x22': [1, 2, 3, 4, 5], 'x23': [1, 2, 3, 4, 5], 'x24': [4], 'x25': [1, 2, 3, 4, 5], 'x31': [1, 2, 3, 4, 5], 'x32': [1, 2, 3, 4, 5], 'x33': [1, 2, 3, 4, 5], 'x34': [1, 2, 3, 4, 5], 'x35': [1, 2, 3, 4, 5], 'x41': [1, 2, 3, 4, 5], 'x42': [1, 2, 3, 4, 5], 'x43': [1, 2, 3, 4, 5], 'x44': [1, 2, 3, 4, 5], 'x45': [1, 2, 3, 4, 5], 'x51': [1, 2, 3, 4, 5], 'x52': [1, 2, 3, 4, 5], 'x53': [1, 2, 3, 4, 5], 'x54': [1, 2, 3, 4, 5], 'x55': [1, 2, 3, 4, 5]}
Vizinhos =  defaultdict(<class 'list'>, {'x11': ['x21', 'x12', 'x31', 'x13', 'x41', 'x14', 'x51', 'x15'], 'x21': ['x11', 'x22', 'x31', 'x23', 'x41', 'x24', 'x51', 'x25'], 'x12': ['x11', 'x22', 'x32', 'x13', 'x42', 'x14', '

In [26]:
dt4={}
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
print('Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas')
start = timeit.default_timer()
r0 = backtracking_search(p4)
stop = timeit.default_timer()
time0 = stop-start
print('Solução:',r0)
print('Time: ', time0)
dt4["Procura com retrocesso, simples, sem inferência nem heurísticas"]=time0

print("\nRetrocesso com forward checking com pré-filtragem por AC3")
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
p_ac3 = AC3(p4)
r1 = backtracking_search(p_ac3,inference = forward_checking)
stop = timeit.default_timer()
time1 = stop-start
print('Solução:',r1)
print('Time: ', time1)
dt4["Procura com retrocesso com forward checking com pré-filtragem por AC3"]=time1

print('\nApliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):')
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
r2 = backtracking_search(p4,select_unassigned_variable = mrv)
stop = timeit.default_timer()
time2 = stop-start
print('Solução:',r2)
print('Time: ', time2)
dt4["Procura com retrocesso com a heurística da variável com menor domínio (mrv)"]=time2

print('\nApliquemos a procura com retrocesso com forward checking e sem heurísticas')
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
r3 = backtracking_search(p4,inference=forward_checking)
stop = timeit.default_timer()
time3 = stop-start
print('Solução:',r3)
print('Time: ', time3)
dt4["Procura com retrocesso com forward checking e sem heurísticas"]=time3

print('\nApliquemos a procura com retrocesso com forward checking e com mrv')
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
r4 = backtracking_search(p4,select_unassigned_variable = mrv,inference=forward_checking)
stop = timeit.default_timer()
time4 = stop-start
print('Solução:',r4)
print('Time: ', time4)
dt4["Procura com retrocesso com forward checking e com mrv"]=time4

print('\nApliquemos a procura com retrocesso com mac')
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
r5 = backtracking_search(p4,inference=mac)
stop = timeit.default_timer()
time5 = stop-start
print('Solução:',r5)
print('Time: ', time5)
dt4["Procura com retrocesso com mac"]=time5

print('\nApliquemos a procura com retrocesso com mac e com mrv')
p4 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
start = timeit.default_timer()
r6 = backtracking_search(p4,select_unassigned_variable = mrv,inference=mac)
stop = timeit.default_timer()
time6 = stop-start
print('Solução:',r6)
print('Time: ', time6)
dt4["Procura com retrocesso com mac e com mrv"]=time6

Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 5, 'x15': 3, 'x21': 3, 'x22': 2, 'x23': 1, 'x24': 4, 'x25': 5, 'x31': 2, 'x32': 3, 'x33': 5, 'x34': 1, 'x35': 4, 'x41': 5, 'x42': 4, 'x43': 3, 'x44': 2, 'x45': 1, 'x51': 1, 'x52': 5, 'x53': 4, 'x54': 3, 'x55': 2}
Time:  2.7653461999999998

Retrocesso com forward checking com pré-filtragem por AC3
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 5, 'x15': 3, 'x21': 3, 'x22': 2, 'x23': 1, 'x24': 4, 'x25': 5, 'x31': 2, 'x32': 3, 'x33': 5, 'x34': 1, 'x35': 4, 'x41': 5, 'x42': 4, 'x43': 3, 'x44': 2, 'x45': 1, 'x51': 1, 'x52': 5, 'x53': 4, 'x54': 3, 'x55': 2}
Time:  0.6982347000000004

Apliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):
Solução: {'x24': 4, 'x15': 3, 'x42': 4, 'x54': 3, 'x33': 5, 'x55': 2, 'x32': 3, 'x25': 5, 'x35': 4, 'x44': 2, 'x51': 1, 'x23': 1, 'x52': 5, 'x22': 2, 'x11': 4, 'x41': 5, 'x13': 2, 'x21': 3, 'x43': 3, '

In [27]:
for item in sorted(dt4, key = dt4.get):
    print (item, dt4[item])

Procura com retrocesso com forward checking e com mrv 0.3314267000000086
Procura com retrocesso com forward checking com pré-filtragem por AC3 0.6982347000000004
Procura com retrocesso com forward checking e sem heurísticas 0.9041131000000178
Procura com retrocesso com mac e com mrv 0.913449700000001
Procura com retrocesso com mac 1.3256474000000367
Procura com retrocesso, simples, sem inferência nem heurísticas 2.7653461999999998
Procura com retrocesso com a heurística da variável com menor domínio (mrv) 464.0683047


#### Futoshiki *4x4*

In [28]:
dt5={}
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
print('Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas')
start = timeit.default_timer()
r0 = backtracking_search(p5)
stop = timeit.default_timer()
time0 = stop-start
print('Solução:',r0)
print('Time: ', time0)
dt5["Procura com retrocesso, simples, sem inferência nem heurísticas"]=time0

print("\nRetrocesso com forward checking com pré-filtragem por AC3")
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
p_ac3 = AC3(p5)
r1 = backtracking_search(p_ac3,inference = forward_checking)
stop = timeit.default_timer()
time1 = stop-start
print('Solução:',r1)
print('Time: ', time1)
dt5["Procura com retrocesso com forward checking com pré-filtragem por AC3"]=time1

print('\nApliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):')
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
r2 = backtracking_search(p5,select_unassigned_variable = mrv)
stop = timeit.default_timer()
time2 = stop-start
print('Solução:',r2)
print('Time: ', time2)
dt5["Procura com retrocesso com a heurística da variável com menor domínio (mrv)"]=time2

print('\nApliquemos a procura com retrocesso com forward checking e sem heurísticas')
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
r3 = backtracking_search(p5,inference=forward_checking)
stop = timeit.default_timer()
time3 = stop-start
print('Solução:',r3)
print('Time: ', time3)
dt5["Procura com retrocesso com forward checking e sem heurísticas"]=time3

print('\nApliquemos a procura com retrocesso com forward checking e com mrv')
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
r4 = backtracking_search(p5,select_unassigned_variable = mrv,inference=forward_checking)
stop = timeit.default_timer()
time4 = stop-start
print('Solução:',r4)
print('Time: ', time4)
dt5["Procura com retrocesso com forward checking e com mrv"]=time4

print('\nApliquemos a procura com retrocesso com mac')
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
r5 = backtracking_search(p5,inference=mac)
stop = timeit.default_timer()
time5 = stop-start
print('Solução:',r5)
print('Time: ', time5)
dt5["Procura com retrocesso com mac"]=time5

print('\nApliquemos a procura com retrocesso com mac e com mrv')
p5 = futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])
start = timeit.default_timer()
r6 = backtracking_search(p5,select_unassigned_variable = mrv,inference=mac)
stop = timeit.default_timer()
time6 = stop-start
print('Solução:',r6)
print('Time: ', time6)
dt5["Procura com retrocesso com mac e com mrv"]=time6

Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x21': 3, 'x22': 2, 'x23': 1, 'x24': 4, 'x31': 2, 'x32': 4, 'x33': 3, 'x34': 1, 'x41': 1, 'x42': 3, 'x43': 4, 'x44': 2}
Time:  0.18788090000003876

Retrocesso com forward checking com pré-filtragem por AC3
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x21': 3, 'x22': 2, 'x23': 1, 'x24': 4, 'x31': 2, 'x32': 4, 'x33': 3, 'x34': 1, 'x41': 1, 'x42': 3, 'x43': 4, 'x44': 2}
Time:  0.1491345000000024

Apliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):
Solução: {'x41': 1, 'x14': 3, 'x32': 4, 'x11': 4, 'x23': 1, 'x42': 3, 'x24': 4, 'x44': 2, 'x12': 1, 'x13': 2, 'x21': 3, 'x33': 3, 'x31': 2, 'x34': 1, 'x43': 4, 'x22': 2}
Time:  1.942036900000005

Apliquemos a procura com retrocesso com forward checking e sem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x21': 3, 'x22': 2, 'x23': 1, 'x24': 4, 'x31': 2, 'x32': 4,

In [29]:
for item in sorted(dt5, key = dt5.get):
    print (item, dt5[item])

Procura com retrocesso com forward checking e com mrv 0.0368773999999803
Procura com retrocesso com forward checking e sem heurísticas 0.11386970000000929
Procura com retrocesso com mac e com mrv 0.11752930000000106
Procura com retrocesso com forward checking com pré-filtragem por AC3 0.1491345000000024
Procura com retrocesso, simples, sem inferência nem heurísticas 0.18788090000003876
Procura com retrocesso com mac 0.309887099999969
Procura com retrocesso com a heurística da variável com menor domínio (mrv) 1.942036900000005


Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

In [30]:
pdim4_1 = futoshiki(4, ['x11 = 4', 'x44 = 1'], ['x12 < x13', 'x13 < x14', 'x33 < x34', 'x42 > x43'])
pdim5_1 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
pdim6_1 = futoshiki(6, ['x16 = 6', 'x45 = 4', 'x53 = 5', 'x56 = 3'], ['x11 < x12', 'x21 < x22', 'x13 < x23', 'x32 > x42', 'x41 < x42', 'x35 > x36', 'x36 > x46', 'x45 < x55', 'x56 < x66', 'x61 < x62', 'x63 > x64'])

pdim4_2 = futoshiki(4, ['x11 = 4', 'x44 = 1'], ['x12 < x13', 'x13 < x14', 'x33 < x34', 'x42 > x43'])
pdim5_2 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
pdim6_2 = futoshiki(6, ['x16 = 6', 'x45 = 4', 'x53 = 5', 'x56 = 3'], ['x11 < x12', 'x21 < x22', 'x13 < x23', 'x32 > x42', 'x41 < x42', 'x35 > x36', 'x36 > x46', 'x45 < x55', 'x56 < x66', 'x61 < x62', 'x63 > x64'])

pdim4_3 = futoshiki(4, ['x11 = 4', 'x44 = 1'], ['x12 < x13', 'x13 < x14', 'x33 < x34', 'x42 > x43'])
pdim5_3 = futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])
pdim6_3 = futoshiki(6, ['x16 = 6', 'x45 = 4', 'x53 = 5', 'x56 = 3'], ['x11 < x12', 'x21 < x22', 'x13 < x23', 'x32 > x42', 'x41 < x42', 'x35 > x36', 'x36 > x46', 'x45 < x55', 'x56 < x66', 'x61 < x62', 'x63 > x64'])

list1 = [pdim4_1, pdim4_2, pdim4_3]
list2 = [pdim5_1, pdim5_2, pdim5_3]
list3 = [pdim6_1, pdim6_2, pdim6_3]

listasproblemas = [list1, list2, list3]

dim = 4
for lista in listasproblemas:
    print("\n \nVamos resolver o problema de dimensao", dim)

    print('Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas')
    p = lista[0]
    start = timeit.default_timer()
    r = backtracking_search(p)
    stop = timeit.default_timer()
    time_sem_inferencia = stop-start
    print('Solução:',r)
    print('Time: ', time_sem_inferencia)
    if time_sem_inferencia > 60:
        print('Solução com tempo superior a 1min')
        break

    print('\nApliquemos a procura com retrocesso com forward checking e sem heurísticas')
    p = lista[1]
    start = timeit.default_timer()
    r = backtracking_search(p,inference=forward_checking)
    stop = timeit.default_timer()
    time_com_inferencia = stop-start
    print('Solução:',r)
    print('Time: ', time_com_inferencia)
    if time_com_inferencia > 60:
        print('Solução com tempo superior a 1min')
        break

    print('\nApliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):')
    p = lista[2]
    start = timeit.default_timer()
    r = backtracking_search(p,select_unassigned_variable = mrv)
    stop = timeit.default_timer()
    time_com_heuristica = stop-start
    print('Solução:',r)
    print('Time: ', time_com_heuristica)
    if time_com_heuristica > 60:
        print('Solução com tempo superior a 1min')
        break
    dim += 1
print("É possível correr os três algoritmos num tempo inferior a 1min até à dimensão", dim-1)


 
Vamos resolver o problema de dimensao 4
Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x21': 1, 'x22': 3, 'x23': 4, 'x24': 2, 'x31': 3, 'x32': 2, 'x33': 1, 'x34': 4, 'x41': 2, 'x42': 4, 'x43': 3, 'x44': 1}
Time:  0.05617419999998674

Apliquemos a procura com retrocesso com forward checking e sem heurísticas
Solução: {'x11': 4, 'x12': 1, 'x13': 2, 'x14': 3, 'x21': 1, 'x22': 3, 'x23': 4, 'x24': 2, 'x31': 3, 'x32': 2, 'x33': 1, 'x34': 4, 'x41': 2, 'x42': 4, 'x43': 3, 'x44': 1}
Time:  0.04756609999998318

Apliquemos a procura com retrocesso com a heurística da variável com menor domínio (mrv):
Solução: {'x44': 1, 'x11': 4, 'x42': 4, 'x34': 4, 'x13': 2, 'x24': 2, 'x43': 3, 'x41': 2, 'x23': 4, 'x33': 1, 'x21': 1, 'x32': 2, 'x14': 3, 'x22': 3, 'x31': 3, 'x12': 1}
Time:  0.5660715999999866

 
Vamos resolver o problema de dimensao 5
Apliquemos a procura com retrocesso, simples, sem inferência nem heurísticas
Sol

## Visualização do problema

In [31]:
# Implemente uma função que permita visualizar o puzzle Futoshiki, antes e depois de resolvido. Compare com a solução obtida pelo seu algoritmo. 
# No caso de não implementar esta função, inclua um screenshot do problema e da sua solução.
def display_futoshiki(n, lista_entradas, lista_desigualdades):
    variaveis = [('x' + str(i) + str(j)) for i in range(1,n+1) for j in range(1,n+1)]
    print('Futoshiki antes de estar resolvido')
    barra = '----' * n
    print( ' '+barra[:-1] +' ')
    for v in variaveis:
        if variaveis.index(v)%n==0:
            print('| ',end='')
        conteudo = ' '
        separacao = ' | '
        for restricoes in lista_desigualdades:
            variavel_esquerda = restricoes.split()[0]
            desigualdade = restricoes.split()[1]
            variavel_direita = restricoes.split()[2]
            if v in variavel_esquerda:
                if v[1] == variavel_direita[1]:
                    separacao = ' ' + desigualdade + ' '
        for restricoes in lista_entradas:
            if v in restricoes.split()[0]:
                conteudo = restricoes.split()[2]
        print(conteudo, end = separacao)
        if variaveis.index(v)%n==n-1:
            linha = ''
            for c in range(1, n+1):
                adicionar_linha = '----'
                for restricoes in lista_desigualdades:
                    esq = restricoes.split()[0]
                    desigualdade = restricoes.split()[1]
                    di = restricoes.split()[2]
                    if int(esq[1]) > int(di[1]):
                        esq, di = di, esq
                        if desigualdade == '<':
                            desigualdade = '>'
                        else:
                            desigualdade = '<'
                    if ('x' + v[1] + str(c)) == esq:
                        if ('x' + str(int(esq[1])+1) + esq[2]) == di:
                            if desigualdade == '<':
                                adicionar_linha = '-' + u"\u2227" + '--'
                            if desigualdade == '>':
                                adicionar_linha = '-' + u"\u2228" + '--'
                linha += adicionar_linha
            print()
            print(' ' + linha[:-1] + ' ')
    r = backtracking_search(futoshiki(n, lista_entradas, lista_desigualdades))
    print('Futoshiki depois de estar resolvido')
    barra = '----' * n
    print( ' '+barra[:-1] +' ')
    for v in variaveis:
        if variaveis.index(v)%n==0:
            print('| ',end='')
        conteudo = ' '
        separacao = ' | '
        for restricoes in lista_desigualdades:
            variavel_esquerda = restricoes.split()[0]
            desigualdade = restricoes.split()[1]
            variavel_direita = restricoes.split()[2]
            if v in variavel_esquerda:
                if v[1] == variavel_direita[1]:
                    separacao = ' ' + desigualdade + ' '
        conteudo = r[v]
        print(conteudo, end = separacao)
        if variaveis.index(v)%n==n-1:
            linha = ''
            for c in range(1, n+1):
                adicionar_linha = '----'
                for restricoes in lista_desigualdades:
                    esq = restricoes.split()[0]
                    desigualdade = restricoes.split()[1]
                    di = restricoes.split()[2]
                    if int(esq[1]) > int(di[1]):
                        esq, di = di, esq
                        if desigualdade == '<':
                            desigualdade = '>'
                        else:
                            desigualdade = '<'
                    if ('x' + v[1] + str(c)) == esq:
                        if ('x' + str(int(esq[1])+1) + esq[2]) == di:
                            if desigualdade == '<':
                                adicionar_linha = '-' + u"\u2227" + '--'
                            if desigualdade == '>':
                                adicionar_linha = '-' + u"\u2228" + '--'
                linha += adicionar_linha
            print()
            print(' ' + linha[:-1] + ' ')

In [32]:
display_futoshiki(5, ['x15 = 3', 'x24 = 4'], ['x11 > x12', 'x14 > x24', 'x15 < x25', 'x22 > x23', 'x31 < x32', 'x32 < x33', 'x43 < x53', 'x54 > x44', 'x55 > x45'])

Futoshiki antes de estar resolvido
 ------------------- 
|   >   |   |   | 3 | 
 -------------∨---∧- 
|   |   >   | 4 |   | 
 ------------------- 
|   <   <   |   |   | 
 ------------------- 
|   |   |   |   |   | 
 ---------∧---∧---∧- 
|   |   |   |   |   | 
 ------------------- 
Futoshiki depois de estar resolvido
 ------------------- 
| 4 > 1 | 2 | 5 | 3 | 
 -------------∨---∧- 
| 3 | 2 > 1 | 4 | 5 | 
 ------------------- 
| 2 < 3 < 5 | 1 | 4 | 
 ------------------- 
| 5 | 4 | 3 | 2 | 1 | 
 ---------∧---∧---∧- 
| 1 | 5 | 4 | 3 | 2 | 
 ------------------- 


In [33]:
display_futoshiki(4, [], ['x12 < x13', 'x11 > x21', 'x21 > x22', 'x23 < x33', 'x31 > x41', 'x42 < x43'])

Futoshiki antes de estar resolvido
 --------------- 
|   |   <   |   | 
 -∨------------- 
|   >   |   |   | 
 ---------∧----- 
|   |   |   |   | 
 -∨------------- 
|   |   <   |   | 
 --------------- 
Futoshiki depois de estar resolvido
 --------------- 
| 4 | 1 < 2 | 3 | 
 -∨------------- 
| 3 > 2 | 1 | 4 | 
 ---------∧----- 
| 2 | 4 | 3 | 1 | 
 -∨------------- 
| 1 | 3 < 4 | 2 | 
 --------------- 
